In [ ]:
import torch
!pip install -q torch-scatter~=2.1.0 torch-sparse~=0.6.16 torch-cluster~=1.6.0 torch-spline-conv~=1.2.1 torch-geometric==2.2.0 -f https://data.pyg.org/whl/torch-{torch.__version__}.html

torch.manual_seed(11)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#leer el archivo fraudTrain.csv
df = pd.read_csv('fraudTrain.csv')

In [ ]:
df.shape

(1296675, 23)

In [ ]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [ ]:
import pandas as pd
import networkx as nx
import torch
from torch_geometric.utils import from_networkx
from torch_geometric.nn import Node2Vec

# Crear un grafo bipartito en NetworkX
B = nx.Graph()

# Agregar nodos de tipo "cc_num" y "merchant"
for index, row in df.iterrows():
    B.add_node(row['cc_num'], bipartite=0)  # Nodos de tarjetas
    B.add_node(row['merchant'], bipartite=1)  # Nodos de comerciantes
    B.add_edge(row['cc_num'], row['merchant'], is_fraud=row['is_fraud'], weight=row['amt'])

# Convertir el grafo NetworkX a un formato compatible con PyTorch Geometric
data = from_networkx(B)

In [ ]:
#Modelo Node2Vec de PyTorch Geometric

model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
                 context_size=10, walks_per_node=10,
                 num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

In [ ]:
# prompt: Calacular los embedding de los nodos

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


for epoch in range(1, 101):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

# Obtener los embeddings de los nodos
with torch.no_grad():
    model.eval()
    z = model()

Epoch: 01, Loss: 9.3359
Epoch: 02, Loss: 7.9458
Epoch: 03, Loss: 6.7598
Epoch: 04, Loss: 5.7509
Epoch: 05, Loss: 4.9093
Epoch: 06, Loss: 4.2278
Epoch: 07, Loss: 3.6701
Epoch: 08, Loss: 3.2176
Epoch: 09, Loss: 2.8439
Epoch: 10, Loss: 2.5428
Epoch: 11, Loss: 2.2998
Epoch: 12, Loss: 2.1111
Epoch: 13, Loss: 1.9569
Epoch: 14, Loss: 1.8363
Epoch: 15, Loss: 1.7434
Epoch: 16, Loss: 1.6687
Epoch: 17, Loss: 1.6096
Epoch: 18, Loss: 1.5642
Epoch: 19, Loss: 1.5280
Epoch: 20, Loss: 1.5006
Epoch: 21, Loss: 1.4774
Epoch: 22, Loss: 1.4586
Epoch: 23, Loss: 1.4444
Epoch: 24, Loss: 1.4305
Epoch: 25, Loss: 1.4208
Epoch: 26, Loss: 1.4128
Epoch: 27, Loss: 1.4050
Epoch: 28, Loss: 1.4002
Epoch: 29, Loss: 1.3959
Epoch: 30, Loss: 1.3907
Epoch: 31, Loss: 1.3869
Epoch: 32, Loss: 1.3833
Epoch: 33, Loss: 1.3816
Epoch: 34, Loss: 1.3798
Epoch: 35, Loss: 1.3771
Epoch: 36, Loss: 1.3755
Epoch: 37, Loss: 1.3730
Epoch: 38, Loss: 1.3728
Epoch: 39, Loss: 1.3711
Epoch: 40, Loss: 1.3702
Epoch: 41, Loss: 1.3692
Epoch: 42, Loss:

In [ ]:
# Obtener los embeddings finales
z = model()
z

Parameter containing:
tensor([[-8.0865e-02, -3.0889e-02, -2.5228e-02,  ...,  1.1169e-01,
         -4.3699e-02,  6.2182e-03],
        [-3.2690e-02,  7.8534e-02, -1.2091e-02,  ...,  5.0762e-02,
         -1.0009e-04,  1.2533e-01],
        [-1.4738e-01, -5.9525e-02, -7.2428e-03,  ...,  5.5871e-02,
          3.0771e-02,  1.1714e-01],
        ...,
        [ 1.8672e-01,  5.7174e-01,  2.9392e-01,  ...,  1.4580e-02,
          1.4812e-01, -9.4122e-01],
        [ 1.8211e-01,  1.1616e-01,  2.8897e-01,  ..., -1.6374e-01,
          9.5418e-02, -3.3067e-01],
        [ 3.5704e-01,  1.5059e-01,  7.4593e-02,  ..., -2.6017e-02,
          1.2968e-02, -2.0739e-01]], device='cuda:0', requires_grad=True)

In [ ]:
len(z)

1676

In [ ]:
#Lista para almacenar los embeddings combinados
combined_embeddings = []

for index, row in df.iterrows():
  source_node_id = list(B.nodes).index(row['cc_num'])
  target_node_id = list(B.nodes).index(row['merchant'])

  # Embeddings del nodo origen y el nodo destino
  source_embedding = z[source_node_id].detach().cpu().numpy()
  target_embedding = z[target_node_id].detach().cpu().numpy()

  # Combinar los embeddings por concatenación
  combined_embedding = (source_embedding + target_embedding) / 2
  combined_embeddings.append(combined_embedding)

df['combined_embedding'] = combined_embeddings

In [ ]:
df['combined_embedding'].head()

,combined_embedding
0,"[-0.05677771, 0.023822661, -0.018659554, -0.00..."
1,"[-0.1454167, -0.17490116, 0.016489804, -0.1093..."
2,"[-0.055364683, -0.058922924, -0.042528406, 0.0..."
3,"[-0.099259056, 0.005834435, -0.060185254, 0.03..."
4,"[-0.052718617, -0.04807537, -0.071084976, -0.0..."


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento, prueba y validación
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

print("Tamaño del conjunto de entrenamiento:", len(train_df))
print("Tamaño del conjunto de prueba:", len(test_df))
print("Tamaño del conjunto de validación:", len(val_df))

Tamaño del conjunto de entrenamiento: 907672
Tamaño del conjunto de prueba: 194502
Tamaño del conjunto de validación: 194501


In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Separar las características (X) y la variable objetivo (y)
X_train = train_df.drop('is_fraud', axis=1)
y_train = train_df['is_fraud']

# Aplicar Random Under-Sampling
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

# DataFrame balanceado
train_df_balanced = pd.concat([X_train_resampled, y_train_resampled], axis=1)

print("Tamaño del conjunto de entrenamiento original:", len(train_df))
print("Tamaño del conjunto de entrenamiento balanceado:", len(train_df_balanced))
print("Distribución de clases en el conjunto de entrenamiento balanceado:", train_df_balanced['is_fraud'].value_counts())

Tamaño del conjunto de entrenamiento original: 907672
Tamaño del conjunto de entrenamiento balanceado: 10442
Distribución de clases en el conjunto de entrenamiento balanceado: is_fraud
0    5221
1    5221
Name: count, dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Datos de entrenamiento
X_train = [embedding for embedding in train_df_balanced['combined_embedding']]
y_train = train_df_balanced['is_fraud']

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Preparar datos de entrenamiento
X_train = []
for index, row in train_df_balanced.iterrows():
    embedding = row['combined_embedding']
    features = [row['amt'], row['zip'], row['lat'], row['long'], row['city_pop'], row['unix_time'], row['merch_lat'], row['merch_long']]
    X_train.append([*embedding, *features])
y_train = train_df_balanced['is_fraud']


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

# Preparar los datos de prueba
X_test = []
for index, row in test_df.iterrows():
    embedding = row['combined_embedding']
    features = [row['amt'], row['zip'], row['lat'], row['long'], row['city_pop'], row['unix_time'], row['merch_lat'], row['merch_long']]
    X_test.append([*embedding, *features])
y_test = test_df['is_fraud']

# Preparar los datos de validación
X_val = []
for index, row in val_df.iterrows():
    embedding = row['combined_embedding']
    features = [row['amt'], row['zip'], row['lat'], row['long'], row['city_pop'], row['unix_time'], row['merch_lat'], row['merch_long']]
    X_val.append([*embedding, *features])
y_val = val_df['is_fraud']


# Predicciones en los conjuntos de datos
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)
y_val_pred = rf_model.predict(X_val)


# Métricas para los tres conjuntos de datos
def calculate_metrics(y_true, y_pred):
  f1 = f1_score(y_true, y_pred)
  accuracy = accuracy_score(y_true, y_pred)
  try:
      auc = roc_auc_score(y_true, y_pred)
  except ValueError:
      auc = 0
  return f1, accuracy, auc

train_f1, train_accuracy, train_auc = calculate_metrics(y_train, y_train_pred)
test_f1, test_accuracy, test_auc = calculate_metrics(y_test, y_test_pred)
val_f1, val_accuracy, val_auc = calculate_metrics(y_val, y_val_pred)

print("Métricas para el conjunto de entrenamiento:")
print(f"  Accuracy: {train_accuracy:.4f}")
print(f"  AUC: {train_auc:.4f}")

print("\nMétricas para el conjunto de prueba:")
print(f"  Accuracy: {test_accuracy:.4f}")
print(f"  AUC: {test_auc:.4f}")

print("\nMétricas para el conjunto de validación:")
print(f"  Accuracy: {val_accuracy:.4f}")
print(f"  AUC: {val_auc:.4f}")

Métricas para el conjunto de entrenamiento:
  Accuracy: 1.0000
  AUC: 1.0000

Métricas para el conjunto de prueba:
  Accuracy: 0.9494
  AUC: 0.8943

Métricas para el conjunto de validación:
  Accuracy: 0.9511
  AUC: 0.8781
